## SILVER TO GOLD LAYER

### Gold Layer - Current Holdings


In [6]:
## Import necessary libraries and utility functions
import pandas as pd
from ETLTools import GlobalPath, Portfolio

In [7]:
# Instantiate GlobalPath
tradehistory_silver_file_path = GlobalPath(
    "DATA/SILVER/TradeHistory/TradeHistory_data.csv"
)
current_holding_records_file_path = GlobalPath(
    "DATA/GOLD/Holdings/CurrentHoldings_data.csv"
)

### Data Processing

- Read and sort trade history data.
- Apply portfolio trade logic.


In [8]:
# Read the CSV file
df_trade_history = pd.read_csv(tradehistory_silver_file_path)

# Filter the DataFrame to include only rows where the segment is 'EQ' (Equity) or 'MF' (Mutual Funds)
df_trade_history = df_trade_history[
    df_trade_history["segment"].isin(["EQ", "MF"])
]

# Convert 'datetime' to datetime type
df_trade_history["datetime"] = pd.to_datetime(df_trade_history["datetime"])

# Sort the DataFrame by 'datetime'
df_trade_history = df_trade_history.sort_values(by="datetime")

print(
    f"Read SILVER Layer trade history data from: {tradehistory_silver_file_path.relative_path()}"
)

Read SILVER Layer trade history data from: DATA/SILVER/TradeHistory/TradeHistory_data.csv


### Portfolio Logic Application

- Instantiate Portfolio and apply trade logic.
- Handle expired stocks


In [9]:
# Apply the trade logic to each row of the DataFrame
# Instantiate the Portfolio object
portfolio = Portfolio()
for record in df_trade_history.astype(str).to_dict(orient="records"):
    portfolio.trade(record)

portfolio.check_expired_stocks()

### Final Processing and Export

- Select and sort relevant columns.
- Save the processed data as a CSV file in the Gold layer.


In [10]:
# Create a DataFrame from the processed data
df_holding_records = pd.DataFrame(portfolio.get_current_holdings())

# Remove '_amount' suffix from column names for clarity
df_holding_records = df_holding_records.rename(
    columns={
        col: col.replace("open_", "") for col in df_holding_records.columns
    }
)
# Sort the DataFrame
df_holding_records = df_holding_records.sort_values(
    by=[
        "segment",
        "exchange",
        "symbol",
        "scrip_name",
        "datetime",
    ]
)

# Round the values in to two decimal places
df_holding_records = df_holding_records.round(2)

# Reset index to ensure it starts from 0
df_holding_records = df_holding_records.reset_index(drop=True)

# Select and reorder the columns for the final DataFrame
df_holding_records = df_holding_records[
    [
        "scrip_name",
        "symbol",
        "exchange",
        "segment",
        "datetime",
        "side",
        "quantity",
        "price",
        "amount",
    ]
]

# Save the final DataFrame to a CSV file
df_holding_records.to_csv(current_holding_records_file_path, index=None)

print("GOLD Layer CSV file for ProfitLoss successfully created at:")
print(current_holding_records_file_path.relative_path())

# Display the DataFrame debugrmation
df_holding_records.info()

GOLD Layer CSV file for ProfitLoss successfully created at:
DATA/GOLD/Holdings/CurrentHoldings_data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   scrip_name  16 non-null     object        
 1   symbol      16 non-null     object        
 2   exchange    16 non-null     object        
 3   segment     16 non-null     object        
 4   datetime    16 non-null     datetime64[ns]
 5   side        16 non-null     object        
 6   quantity    16 non-null     float64       
 7   price       16 non-null     float64       
 8   amount      16 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 1.3+ KB
